In [1]:
import sys
%matplotlib widget

from msc_thesis_functions import *
del sys.modules['msc_thesis_functions']
from msc_thesis_functions import *

from scipy.stats import linregress
import seaborn as sns

import cmocean as cmo
import cmasher as cmr
import pyproj
import verde as vd

import h5py
import os
import numpy as np
import random
from haversine import haversine, Unit, inverse_haversine

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

from pydlc import dense_lines

import pandas as pd
import geopandas as gpd
import xarray as xr
from datetime import datetime, timedelta

import cartopy
from cartopy.mpl import geoaxes
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.geodesic import Geodesic

import shapely
from shapely.geometry import Polygon, Point, MultiPoint, LineString
from shapely.geometry import box

from tqdm.notebook import trange, tqdm
from time import sleep
import rioxarray

#Variogram 
import gstools as gs
# import xdem
from xdem.terrain import get_terrain_attribute

# import skgstat as skg
# from geopy.distance import vincenty
from scipy.signal import find_peaks
from scipy.spatial.distance import pdist, squareform

#Kriging (pykrige, gstoools, skg)
import pykrige.kriging_tools as kt

from pykrige.ok import OrdinaryKriging
del sys.modules['pykrige.ok']
from pykrige.ok import OrdinaryKriging

from pykrige.uk import UniversalKriging


# Building a 'Blues' colormap where the lower end (white) is transperant
ncolors = 256
color_array = plt.get_cmap('Blues')(range(ncolors))
color_array[:,-1] = np.linspace(0.0,1.0,ncolors)
map_object = LinearSegmentedColormap.from_list(name='Blues_alpha',colors=color_array)

# register this new colormap with matplotlib
# plt.colormaps.register(cmap=map_object)

from pyfonts import load_font

# font = load_font(
#    "https://github.com/google/fonts/blob/main/ofl/roboto/Roboto[wdth,wght].ttf?raw=true"
# )

from pyproj import Transformer
transformer = Transformer.from_crs(4326, 3413, always_xy=True)

pd.options.mode.chained_assignment = None  # default='warn'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from IPython.display import clear_output


In [2]:
def get_level_ice_elevation(elevs):
    df = pd.DataFrame({'elevation': elevs})
        
    df['elev_quantile'] = pd.qcut(elevs, q=100,   labels=False, duplicates='drop')
    min_ind = (df.groupby('elev_quantile')['elevation'].mean().diff().argmin() - 10 , df.groupby('elev_quantile')['elevation'].mean().diff().argmin() + 10)
    level_ice_elevation = df.groupby('elev_quantile')['elevation'].mean().loc[min_ind[0]:min_ind[1]].mean()
    return level_ice_elevation

    

### Split ATM into multiple dfs

In [3]:
elevation_threshold = .2
length = 25000

output_path = '/Users/torka/Library/CloudStorage/OneDrive-Personal/MarineSciences/MasterThs-T/Data/OIB/ATM/2014/ATM_levelled_classified'
atm_path = '/Users/torka/Library/CloudStorage/OneDrive-Personal/MarineSciences/MasterThs-T/Data/OIB/ATM/2014'

files = []
for root, dirs, filenames in os.walk(atm_path):
    for filename in filenames:
        if filename.endswith('.h5'):
            files.append(os.path.join(root, filename))

files = np.sort(files)
# print(files)

writing in smaller block size to make it easier to subset later in the snow radar matching. AND FOR LEVEL ICE SURFACE (ssh changes)

In [5]:
polys = []
ks = []
dates = []
iss = []

for k, file in enumerate(tqdm(files)):

    date_str = file.split('_')[1]
    f = h5py.File(os.path.join(atm_path, file), 'r')
    elev = f['elevation']
    lon = f['longitude']
    lat = f['latitude']
    time = f['instrument_parameters']['time_hhmmss']
    
    df_ATM = pd.DataFrame({'time':time, 'lon':lon, 'lat':lat, 'elev':elev})
    df_ATM['x'], df_ATM['y'] = transformer.transform(df_ATM['lon'].values, df_ATM['lat'].values)

    blocks = np.arange(0, len(df_ATM) + length, length)

    for i, l in enumerate(blocks[:-1]):
        subset =  df_ATM[l:blocks[i+1]].copy()
        level_ice_elevation = get_level_ice_elevation(subset['elev'])
        subset['elev_levelled'] = subset['elev'] - level_ice_elevation
        subset['classes'] = subset['elev_levelled'].apply(lambda x: 1 if x > elevation_threshold else 0)
        
        # subset = subset.loc[subset['classes'] == 1] 
        filename = f'{date_str}_ATM_levelled_{k}_{i}.csv'
        subset.to_csv(os.path.join(output_path, filename), columns=['time','x','y','elev_levelled','classes'], index=False)
        
        bbox = (subset['x'].min(), subset['y'].min(), subset['x'].max(), subset['y'].max())
        
        polys.append(bbox)
        dates.append(date_str)
        ks.append(k)
        iss.append(i)

df_poly = pd.DataFrame({'date':dates, 'bbox':polys, 'k':ks, 'i':iss})
df_poly.to_csv(os.path.join(output_path, 'polygons.csv'), index=False)

  0%|          | 0/55 [00:00<?, ?it/s]

In [6]:
for k, file in enumerate(files):

    date_str = file.split('_')[1]
    f = h5py.File(os.path.join(atm_path, file), 'r')
    elev = f['elevation']
    lon = f['longitude']
    lat = f['latitude']
    time = f['instrument_parameters']['time_hhmmss']
    df_ATM = pd.DataFrame({'time':time, 'lon':lon, 'lat':lat, 'elev':elev})

    df_ATM['x'], df_ATM['y'] = transformer.transform(df_ATM['lon'].values, df_ATM['lat'].values)

    level_ice_elevation = get_level_ice_elevation(df_ATM['elev'])
    df_ATM['elev_levelled'] = df_ATM['elev'] - level_ice_elevation
    
    blocks = np.arange(0, len(df_ATM) + length, length)
    print(f'Length: {len(df_ATM)}, Blocks: {len(blocks)}')

    for i, l in enumerate(blocks[:-1]):
        
        filename = f'{date_str}_ATM_gridded_{k}_{i}.nc'
        
        if filename in os.listdir(output_path) or date_str == '20160419':
            if filename in os.listdir(output_path):
                print(f'File already exists: {filename}')
            if date_str == '20160419':
                print(f'{date_str}, not considered...')
            continue
        
        else:
            print(f'Date: {date_str}, {i}')
                
            subset =  df_ATM[l:blocks[i+1]].copy()
            print(f'{l} to {blocks[i+1]}...')
            
            # compute gridded elevations
            blockmean = vd.BlockMean(spacing=spacing, center_coordinates=True)
            block_coordinates, block_elev, block_weights = blockmean.filter(
                coordinates=(subset['x'], subset['y']),
                data=subset['elev_levelled'],
            )
            print(f'Block mean done...')
            
            cubic = vd.KNeighbors()
            cubic.fit(
                coordinates=(block_coordinates[0], block_coordinates[1]),
                data=block_elev
                )

            grid = cubic.grid(spacing=spacing)
            print(f'Grid done...')
            
            mask = vd.distance_mask(
                (subset['x'], subset['y']),
                maxdist=spacing * 3,
                grid=grid
            )
            mask = mask.rio.write_crs("EPSG:3413")
            mask = mask.rename_vars({'scalars': 'elevation'})
            print(f'Masking done...')
            
            # calculate dem attributes using xdem
            slope = get_terrain_attribute(mask['elevation'].values, ["slope"], resolution=spacing, edge_method='nearest')
            roughness = get_terrain_attribute(mask['elevation'].values, ["roughness"], resolution=spacing, edge_method='nearest')

            mask['slope'] =  (('northing', 'easting'), slope)
            mask['roughness'] =  (('northing', 'easting'), roughness)

            # recompute level ice elevation, because of interpolation
            # (should not be much different - ~cm differences)
            level_ice = get_level_ice_elevation(mask['elevation'].values.flatten())
            classes = xr.where(mask['elevation'] > level_ice + elevation_threshold , 1, 0 )
            
            bbox_xy = (mask['easting'].min().values, mask['northing'].min().values, mask['easting'].max().values, mask['northing'].max().values)
            
            mask['classes'] = classes
            mask = mask.assign_attrs(grid_spacing=spacing,
                                    level_ice=level_ice,
                                    classes_threshold=elevation_threshold,
                                    bbox_xy = bbox_xy
                                    )

            mask.to_netcdf(os.path.join(output_path, filename))
            print(f'Data written to file: {filename}')
            
            # Clear the output after each inner loop
            clear_output(wait=True)

Date: 20160420, 5
250000 to 300000...
Block mean done...
Grid done...
Masking done...


KeyboardInterrupt: 